# Getting started with the Depolarizing Plugin

In this repository, we supply a special myQLM plugin, dubbed `DepolaringPlugin`, to perform basic noisy simulations.

When used to build a stack with a perfect QPU, it is equivalent to a noisy QPU with a depolarizing noise model.

Here, we define the depolarizing noise model by its action on the density matrix:

$$\mathcal{E}(\rho) = (1 - p) \rho + \frac{p}{4^{n_\mathrm{qbits}}-1}\sum_{k = 1}^{4^{n_\mathrm{qbits}}} P_k \rho P_k$$

where $\lbrace P_k, k = 0 \dots 4^{n_\mathrm{qbits}} \rbrace$ denotes the set of all products of Pauli matrices (including the identity) for $n_\mathrm{qubits}$ qubits. By convention, $P_0 = I_{2^{n_\mathrm{qbits}}}$.

Physically, such a channel leaves the density matrix constant with probability $1-p$, and applies a random generalized Pauli error to it with probability $p$. It is often used in proofs of quantum error correction algorithms.

Let us learn how to use it. 

We first build a circuit, and execute it on a perfect QPU for a reference:

In [1]:
from qat.lang.AQASM import H, X, RX, Program, CNOT
from qat.qpus import get_default_qpu

prog = Program()
reg = prog.qalloc(2)
prog.apply(H, reg[0])
prog.apply(X, reg[1])
prog.apply(CNOT, reg)
prog.apply(RX(0.4), reg[0])

circ = prog.to_circ()

qpu = get_default_qpu()
res = qpu.submit(circ.to_job())
for sample in res:
    print(sample.state, sample.probability)

|00> 0.019734751499278724
|01> 0.4802652485007212
|10> 0.4802652485007212
|11> 0.019734751499278724


## With depolarizing plugin

The depolarizing plugin models errors in the following way: each perfect gate, represented by the quantum channel

$$\mathcal{U}(\rho) = U \rho U^\dagger,$$

is replaced by

$$\tilde{\mathcal{U}}(\rho) = \mathcal{E} \circ \mathcal{U} (\rho),$$

namely each gate is followed by a depolarizing channel. Since, in actual hardware, one-qubit and two-qubit gates have very different error rates (they typically differ by an order of magnitude), we use depolarizing channels with different depolarization probabilities for one and two qubit gates. These probabibilities are set using the parameters ``prob_1qb`` and ``prob_2qb``, respectively.

Internally, the simulation of such a channel is made probabilistically using a method similar to the quantum trajectories approach. The number of stochastic samples is set by the ``n_samples`` parameter. Because of the statistical nature of the method, the probabilities obtained for the final state may vary from one run to the other. To reduce this variability, you need to increase the number of samples, at the expense of simulation speed.

In [2]:
from depolarizing_plugin import DepolarizingPlugin
prob_1q = 0.005
prob_2q = 0.05

depol = DepolarizingPlugin(prob_1qb=prob_1q, prob_2qb=prob_2q, n_samples=5000)
res = (depol|qpu).submit(circ.to_job())
for sample in res:
    print(sample.state, sample.probability)

|00> 0.035116470099126926
|01> 0.46488352990088144
|10> 0.46488352990088144
|11> 0.035116470099126926


You can see that the final state distribution is different from the state distribution obtained with a perfect QPU.

### Same computation, with qat.noisy simulator (QLM only)

If you have access to a QLM (as opposed to myQLM only), you can use more powerful simulators to simulate noisy QPUs:

In [3]:
from qat.qpus import NoisyQProc
from qat.quops import make_depolarizing_channel

depol1 = make_depolarizing_channel(prob_1q, 1, 'equal_probs')
depol2 = make_depolarizing_channel(prob_2q, 2, 'equal_probs')

from qat.hardware import DefaultHardwareModel
gate_noise_dict = {}
for gname in ["X", "H"]:
    gate_noise_dict[gname] = lambda: depol1 
for gname in ["RX"]:
    gate_noise_dict[gname] = lambda _: depol1 
for gname in ["CNOT"]:
    gate_noise_dict[gname] = lambda: depol2
hw_model = DefaultHardwareModel(gate_noise=gate_noise_dict)
noisy_qpu = NoisyQProc(hardware_model=hw_model)

res = noisy_qpu.submit(circ.to_job())
for sample in res:
    print(sample.state, sample.probability)

|00> 0.034912335692159735
|01> 0.4650876643078399
|10> 0.4650876643078399
|11> 0.034912335692159735
